In [1]:
import sys
import geopandas
import pandas as pd
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')

## load GIS vector data

In [2]:
wo_geo = geopandas.read_file('/home/philipp/Data/edin_diss/GIS_wo/wo_2016_2020.gdb', layer='Stichtag_20160101')

In [33]:
wo_geo.head()

,FORSTBETRIEB_NR,REVIER_NR,ABTEILUNG,UNTERABTEILUNG,TEILFLAECHE,FARBCODE,LINKID,ID,VERWALTUNGSBLOCK,Erstellungsdatum,Datenaktualität,SHAPE_Length,SHAPE_Area,geometry,WO
4,171,1,506,3,1,00000000003,{E77BB096-11C9-4939-9331-B4B305C4096A},1450402,529,2020-11-20T08:40:34,2016-01-01T00:00:00,2402.546849,11624.655500,"MULTIPOLYGON (((617139.640 493799.045, 617151....",171150631
5,171,1,506,9,2,00000000009,{99BC61C0-CE00-48BB-9140-A0C6D3EB5EA1},1450449,529,2020-11-20T08:40:34,2016-01-01T00:00:00,381.617564,3111.435375,"MULTIPOLYGON (((616697.800 493322.945, 616686....",171150692
6,171,1,506,A,0,14000000000,{D5C7009D-5ED1-44F3-9A10-2EA9AC58094E},1450404,529,2020-11-20T08:40:34,2016-01-01T00:00:00,1367.653182,93101.347400,"MULTIPOLYGON (((617710.960 493854.385, 617742....",1711506A0
7,171,1,506,B,1,28000400000,{E3993FFC-AC51-468D-9B23-A733AF0079C9},1450511,529,2020-11-20T08:40:34,2016-01-01T00:00:00,241.038183,611.014350,"MULTIPOLYGON (((617347.760 493783.145, 617356....",1711506B1
8,171,1,506,B,1,28000400000,{EA311B6D-6915-49CE-B17A-43499CA9077D},1450405,529,2020-11-20T08:40:34,2016-01-01T00:00:00,723.923359,11906.891550,"MULTIPOLYGON (((617454.430 493486.115, 617421....",1711506B1


In [4]:
wo_geo.crs

{'init': 'epsg:31287'}

In [5]:
# filter abteilung -1
wo_geo = wo_geo.loc[wo_geo['ABTEILUNG'] != -1,:]

In [6]:
# create unique ID WO
wo_geo['WO'] = wo_geo['FORSTBETRIEB_NR'].astype(str) + \
wo_geo['REVIER_NR'].astype(str) + \
wo_geo['ABTEILUNG'].astype(str) + \
wo_geo['UNTERABTEILUNG'] + \
wo_geo['TEILFLAECHE'].astype(str)

## load SAP tax data

In [ ]:
# get infromation about which Teiloperate were active in a giveb year

In [19]:
# set year
year = 2016

path_sap_info = '/home/philipp/Data/edin_diss/SAP_tax/edin_meta_data.xlsx'
sap_info = pd.read_excel(path_sap_info)

In [8]:
sap_info.head()

,FB,FR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,171,1,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
1,171,2,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
2,171,3,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
3,171,4,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
4,171,5,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356


In [16]:
# array with all TOs in a given year
tos = sap_info.loc[:,year].unique()

In [17]:
tos

array([1030, 1048, 1049, 1053, 1052, 1100, 1050, 1051, 1083, 1042, 1073,
       1092, 1074, 1075, 1151, 1194, 1086, 1043, 1144, 1197, 1055, 1057,
       1085, 1056, 1140, 1061, 1065, 1066, 1067, 1088, 1044, 1196, 1283,
       1031, 1041, 1079, 1093, 1045, 1081, 1082, 1069, 1070, 1071, 1156,
       1250])

In [28]:
# set sap tax path directory
path_sap_tax_dir = '/home/philipp/Data/edin_diss/SAP_tax'

for to in tos:
    # get fb
    fb = sap_info.loc[sap_info[year] == to, 'FB'].unique()[0]
    # create path to file
    path_sap_tax_file = path_sap_tax_dir + '/' + str(fb) + '/TO_' + str(to) + '.XLS'
    
    print(path_sap_tax_file)


'''
wo_sap = pd.read_csv(path_sap_tax_file, 
                     sep='\t', 
                     encoding = "ISO-8859-1", 
                     decimal=',', 
                     error_bad_lines=False) '''

/home/philipp/Data/edin_diss/SAP_tax/171/TO_1030.XLS
/home/philipp/Data/edin_diss/SAP_tax/171/TO_1048.XLS
/home/philipp/Data/edin_diss/SAP_tax/171/TO_1049.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1053.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1052.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1100.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1050.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1051.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1083.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1042.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1073.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1092.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1074.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1075.XLS
/home/philipp/Data/edin_diss/SAP_tax/174/TO_1151.XLS
/home/philipp/Data/edin_diss/SAP_tax/174/TO_1194.XLS
/home/philipp/Data/edin_diss/SAP_tax/175/TO_1086.XLS
/home/philipp/Data/edin_diss/SAP_tax/175/TO_1043.XLS
/home/philipp/Data/edin_diss/SAP_tax/175/TO_11

'\nwo_sap = pd.read_csv(path_sap_tax_file, \n                     sep=\'\t\', \n                     encoding = "ISO-8859-1", \n                     decimal=\',\', \n                     error_bad_lines=False) '

In [ ]:
'/home/philipp/Data/edin_diss/SAP_tax/171/TO_1030.XLS'

In [29]:
wo_sap = pd.read_csv('/home/philipp/Data/edin_diss/SAP_tax/171/TO_1030.XLS', 
                     sep='\t', 
                     encoding = "ISO-8859-1", 
                     decimal=',', 
                     error_bad_lines=False)

/home/philipp/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (29,127,129,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
# create unique ID WO
wo_sap['WO'] = wo_sap['Forstbetrieb'].astype(str) + \
wo_sap['Forstrevier'].astype(str) + \
wo_sap['Abteilung'].astype(str) + \
wo_sap['Unterabteil.'] + \
wo_sap['Teilfl.'].astype(str)

In [34]:
wo_sap.head()

,Merkmalausprägung,AuswKatTyp,Teiloperats-ID,Forstbetrieb,Debitor,TO-Bezeichnung,Status,Beg. Laufzeit,Ende Laufzeit,Operat-ID,...,Zeile2,Zeile3,Zeile4,Zeile5,Zeile6,Zeile7,Zeile8,Bestockungsziel,Flächenanteil,WO
0,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,BE Im N-TL gelichtet und winddurchrissen mit ...,diese hier älter (+5 J); ein Einzel-WW; im N-T...,vergrast und leicht verkrautet,MA BZ: 5LA 5BU,NaN,NaN,NaN,NaN,0.0,1711648B1
1,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1711648B1
2,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1711648B1
3,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1711648B1
4,0,NaN,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1711648B1


In [36]:
# stoe

# filter data
wo_sap.loc[wo_sap['Best.-Schicht'] == 0, ['WO', 'Forstbetrieb', 'Forstrevier', 'Abteilung', 'Unterabteil.', 'Teilfl.', 'Beg. Laufzeit', 'Umtriebszeit', 'Nebengrund Art', 'Ertragssituation', 'Bewirtschaftungsform', 'Schutzwaldkategorie', 'Seehöhe', 'Exposition', 'Neigung', 'Standorteinheit', 'Vegetationstyp', 'Wuchsgebiet']]

,WO,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Beg. Laufzeit,Umtriebszeit,Nebengrund Art,Ertragssituation,Bewirtschaftungsform,Schutzwaldkategorie,Seehöhe,Exposition,Neigung,Standorteinheit,Vegetationstyp,Wuchsgebiet
0,1711648B1,171,1,648,B,1,01.01.2009,120,NaN,I,W,NaN,400,SO,27,87.0,BW,5.1
6,1711652C1,171,1,652,C,1,01.01.2009,120,NaN,I,W,NaN,400,SW,27,87.0,WW,5.1
12,1711736B4,171,1,736,B,4,01.01.2009,120,NaN,I,W,NaN,300,S,18,87.0,WW,5.1
21,1711658A1,171,1,658,A,1,01.01.2009,120,NaN,I,W,NaN,300,SW,18,87.0,WW,5.1
27,1711506D1,171,1,506,D,1,01.01.2009,120,NaN,I,W,NaN,300,NO,27,88.0,WW,5.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30998,1714214C2,171,4,214,C,2,01.01.2009,100,NaN,I,W,NaN,400,W,9,86.0,KV,4.2
31007,1714341A2,171,4,341,A,2,01.01.2009,120,NaN,I,W,NaN,300,N,9,86.0,KV,5.1
31013,1714336F2,171,4,336,F,2,01.01.2009,120,NaN,I,W,NaN,400,SO,9,96.0,K,5.1
31022,171469B3,171,4,69,B,3,01.01.2009,120,NaN,I,W,NaN,300,NW,27,86.0,BW,5.1


In [44]:
# wood volume

# filter data
wo_sap_v = wo_sap.loc[wo_sap['Best.-Schicht.1'] > 0, ['WO', 'Vorrat / ha', 'Laubholzvorrat / ha',
       'Nadelholzvorrat / ha', 'Vorrat am Ort', 'Laubholzvorrat Ort',
       'Nadelholzvorrat Ort',]]

# group by WO (ID) and sum all values
wo_sap_v = wo_sap_v.groupby(['WO']).sum().reset_index()

wo_sap_v

In [47]:
# wood cuts

# filter data
wo_sap_ma = wo_sap.loc[wo_sap['Nutzungsnummer'] > 0, ['WO', 'Maßnahmenart', 'Massnahmengruppe', 'Angriffsfläche', 'Nutzung LH', 'Nutzung NH', 'Nutzungssumme', 'Nutzdringlichkeit', 'Bewpfl.', 'Zeitpunkt', 'Rückungsart', 'Schlägerungsart']]
wo_sap_ma

,WO,Maßnahmenart,Massnahmengruppe,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart
5,1711648B1,RM,EN,4.0,1600.0,0.0,1600.0,2,3,2,30.0,1.0
10,1711652C1,DF,VN,3.0,100.0,0.0,100.0,2,1,2,30.0,1.0
11,1711652C1,ZE,EN,0.0,30.0,20.0,50.0,3,1,2,30.0,1.0
20,1711736B4,JP,WP,3.5,0.0,0.0,0.0,1,1,1,NaN,NaN
26,1711658A1,DF,VN,4.0,200.0,0.0,200.0,2,1,2,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
30997,171459C3,UE,EN,0.0,50.0,10.0,60.0,1,1,2,30.0,1.0
31006,1714214C2,DE,VN,0.2,0.0,5.0,5.0,1,1,2,30.0,1.0
31012,1714341A2,DF,VN,1.1,40.0,0.0,40.0,2,1,2,30.0,1.0
31036,1714201A4,UE,EN,0.0,0.0,5.0,5.0,1,1,2,30.0,1.0
